In [1]:
# When run, this will acquire new data from Yahoo Finance every 1 second,
# and add it to the Database. The program will run until stopped.

In [2]:
import json
import urllib.request as urlreq
import time, datetime
import pandas as pd
import codecs
import sqlite3 as sq

In [3]:
def get_stock_list(fname):
    """Open .csv file containing list of stocks, and their weights in the ETF"""
    stock_list = pd.read_csv(fname)
    return stock_list

In [4]:
def get_quotes(stock_list):
    """Get quote for each stock in list, return list of responses from the API"""
    url = "http://finance.google.com/finance/info?client=ig&q="
    for stock in stock_list:
        url = url + '{},'.format(stock)
    try:
        raw_data = urlreq.urlopen(url).read().decode('utf8')
    except HTTPError:
        print('Error in API response when fetching', url)
        pass

    return json.loads(raw_data[3:])

In [5]:
def calc_inav(stock_list, dow_divisor):
    """Calculate Indicative Net Asset Value of DJT. Returns calculated value, and value for
    the DJT received from API call"""
    quotes = get_quotes(stock_list['Ticker'])

    inav_df = stock_list.set_index('Ticker')

    for i, stock in enumerate(quotes):
        inav_df.set_value(stock['t'], 'Last_Price', float(stock['l'].replace(',','')))

    inav = inav_df[(inav_df.index != 'DJT')]['Last_Price'].sum(axis=0) / dow_divisor

    market_index = inav_df[(inav_df.index == 'DJT')]['Last_Price'].iloc[0]

    output = {'time': datetime.datetime.now(), 'inav': inav, 'market_index': market_index}

    return output

In [6]:
def connect_to_db():
    """Connect to Database"""
    db = sq.connect('mydb')
    cursor = db.cursor()
    
    return db  

In [7]:
def to_db(con, data):
    """Write data to Database"""
    data.to_sql("data", con, if_exists="append")   

In [8]:
def exit_program(con, values_df):
    """Exit program, commit changes to Database"""
    print('Exiting...')
    # print(values_df)
    con.commit()

    # Exit program:
    exit()

In [9]:
def run_app(stock_list):
    """Will run the program continuously until user exits, getting new data every 1 secs.
    Then, will output summary of data, and save to .csv"""
    
    try:
        values_df = pd.DataFrame()
        
        # Connect to Database, to save new data:
        con = connect_to_db()
        
        while True:
            
            values = calc_inav(stock_list, dow_divisor)
            temp_df = pd.DataFrame([values]).set_index('time')
            temp_df['difference'] = temp_df['inav'] - temp_df['market_index']
            
            # Add new data to existing master DataFrame:
            values_df = pd.concat([values_df, temp_df])
            
            # Save to Database:
            to_db(con, temp_df)
            
            # Wait to get new data:
            time.sleep(1)            

    except KeyboardInterrupt:
        exit_program(con, values_df)

In [10]:
if __name__ == '__main__':

    '''DOW divisor source: http://online.barrons.com/mdc/public/page/9_3022-djiahourly.html?mod=mdc_h_usshl'''
    dow_divisor = 0.16416809180007

    stock_list = get_stock_list('DJT_Component_Weights.csv')

    """Runs the program continuously until exit"""
    run_app(stock_list)

Exiting...
